In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir('/content/drive/My Drive/CA684_Assignment')

In [0]:
!pwd

/content/drive/.shortcut-targets-by-id/1UWIdExqb5P94RLqkaiODcwFC2v7g3lB2/CA684_Assignment


In [0]:
!ls

Additional_Resources  Complete_Sources	Test-set
Assignment_Overview   Dev-set		Tutorials


In [0]:
!pip install pyprind

In [0]:
import pandas as pd
import keras
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
# Import EarlyStopping
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download()
nltk.download('punkt')

import numpy as np
import pyprind
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [0]:
# captions and labels
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load dev set captions
cap_path = '/content/drive/My Drive/CA684_Assignment/Dev-set/Captions/dev-set_video-captions.txt'
df_captions=read_caps(cap_path)

# load dev set ground truth values
label_path = './'
labels=pd.read_csv(label_path+'Dev-set/Ground-truth/ground-truth.csv')


In [0]:
#for test-set



#os.chdir("/content/drive/My Drive/CA684_Assignment_Data/Test-set/")



In [0]:
!pwd

/content/drive/.shortcut-targets-by-id/1UWIdExqb5P94RLqkaiODcwFC2v7g3lB2/CA684_Assignment


In [0]:
!ls

Additional_Resources  Complete_Sources	Test-set
Assignment_Overview   Dev-set		Tutorials


In [0]:

# test-set`
df_captions_test = pd.read_csv('/content/drive/My Drive/CA684_Assignment/Test-set/Captions_test/test-set-1_video-captions.txt', sep='\t',names = ['video','caption'])
df_captions_test.head()




,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


In [0]:
labels_test = pd.read_csv('/content/drive/My Drive/CA684_Assignment/Test-set/Ground-truth_test/ground_truth_template.csv')
labels_test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


In [0]:
df_captions_test['video'].head()


0    video7494.webm
1    video7495.webm
2    video7496.webm
3    video7497.webm
4    video7498.webm
Name: video, dtype: object

In [0]:
df_captions_dev = df_captions.merge(labels,on=["video"],how="inner")
df_captions_dev.head()

,video,caption,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,blonde-woman-is-massaged-tilt-down,0.924,34,0.846,13
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot,0.923,33,0.667,12
2,video6.webm,khr-gangsters,0.863,33,0.700,10
3,video8.webm,medical-helicopter-hovers-at-airport,0.922,33,0.818,11
4,video10.webm,couple-relaxing-on-picnic-crane-shot,0.950,34,0.900,10


In [0]:

def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [0]:
#Loading HMP Features
df_hmp_dev = pd.DataFrame(columns = ['video', 'arrayInfo'])

dir_hmp_dev = '/content/drive/My Drive/CA684_Assignment/Dev-set/HMP/'

for filename in os.listdir(dir_hmp_dev):
    if filename.endswith(".txt"):
        path = os.path.join(dir_hmp_dev, filename)
        array = read_HMP(path)
        fileName= filename.replace(".txt",".webm")
        df_hmp_dev = df_hmp_dev.append({'video': fileName, 'arrayInfo': array}, ignore_index=True)
    else:
        break

In [0]:
df_dev = df_captions_dev.merge(df_hmp_dev,on=["video"],how="inner")
df_dev.columns
df_dev.head()

,video,caption,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,arrayInfo
0,video3.webm,blonde-woman-is-massaged-tilt-down,0.924,34,0.846,13,"[0.125563, 0.024036, 0.000314, 0.0, 0.015864, ..."
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot,0.923,33,0.667,12,"[0.007526, 0.001421, 6.8e-05, 0.0, 0.001184, 0..."
2,video6.webm,khr-gangsters,0.863,33,0.700,10,"[0.109584, 0.018978, 0.000289, 0.0, 0.008774, ..."
3,video8.webm,medical-helicopter-hovers-at-airport,0.922,33,0.818,11,"[0.120431, 0.013561, 0.000277, 0.0, 0.018974, ..."
4,video10.webm,couple-relaxing-on-picnic-crane-shot,0.950,34,0.900,10,"[0.005026, 0.001356, 5.5e-05, 0.0, 0.000665, 2..."


In [0]:
df_hmp_dev

,video,arrayInfo
0,video6634.webm,"[0.012099, 0.001466, 1.7e-05, 0.0, 0.000683, 7..."
1,video6633.webm,"[0.07817, 0.013597, 0.000122, 0.0, 0.00559, 7...."
2,video6639.webm,"[0.112506, 0.014053, 0.000263, 0.0, 0.043205, ..."
3,video6635.webm,"[0.066072, 0.012682, 0.000283, 0.0, 0.006425, ..."
4,video6637.webm,"[0.03036, 0.005834, 0.000169, 0.0, 0.006578, 0..."
...,...,...
5995,video2126.webm,"[0.010845, 0.002114, 7.5e-05, 0.0, 0.001033, 4..."
5996,video2124.webm,"[0.015527, 0.003654, 0.000133, 0.0, 0.003222, ..."
5997,video2121.webm,"[0.007801, 0.003515, 0.000265, 0.0, 0.00144, 0..."
5998,video2117.webm,"[0.012784, 0.002923, 3.2e-05, 0.0, 0.000598, 1..."


In [0]:
# test set
df_hmp_test = pd.DataFrame(columns = ['video', 'arrayInfo'])

dir_hmp_test = '/content/drive/My Drive/CA684_Assignment/Test-set/HMP_test'

for filename in os.listdir(dir_hmp_test):
    if filename.endswith(".txt"):
        path = os.path.join(dir_hmp_test, filename)
        array = read_HMP(path)
        fileName= filename.replace(".txt",".webm")
        df_hmp_test = df_hmp_test.append({'video': fileName, 'arrayInfo': array}, ignore_index=True)
    else:
        break

In [0]:
df_hmp_test.head()

,video,arrayInfo
0,video8763.webm,"[0.00206, 0.000583, 1.1e-05, 0.0, 0.000395, 3...."
1,video8758.webm,"[0.015623, 0.005571, 0.000267, 0.0, 0.003743, ..."
2,video8768.webm,"[0.00533, 0.001166, 2e-06, 0.0, 0.000927, 1.1e..."
3,video8764.webm,"[0.040748, 0.016237, 0.000303, 0.0, 0.007668, ..."
4,video8760.webm,"[0.056045, 0.012873, 0.000278, 0.0, 0.006645, ..."


In [0]:
df_test = df_captions_test.merge(df_hmp_test,on=["video"],how="inner")
df_test.columns
df_test.head()

,video,caption,arrayInfo
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks,"[0.022039, 0.005264, 9.8e-05, 0.0, 0.003689, 0..."
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...,"[0.004694, 0.001934, 4e-05, 0.0, 0.001226, 5.8..."
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...,"[0.02653, 0.006208, 5.8e-05, 0.0, 0.00228, 3.1..."
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...,"[0.045109, 0.008317, 0.000151, 0.0, 0.005598, ..."
4,video7498.webm,slow-motion-of-photographer-in-national-park,"[0.034691, 0.008619, 0.000412, 0.0, 0.028355, ..."


In [0]:
# Tokenizing words in caption
#test set
df_dev['caption'].apply(nltk.word_tokenize)
df_dev['caption'].head()

0                   blonde-woman-is-massaged-tilt-down
1    roulette-table-spinning-with-ball-in-closeup-shot
2                                        khr-gangsters
3                 medical-helicopter-hovers-at-airport
4                 couple-relaxing-on-picnic-crane-shot
Name: caption, dtype: object

In [0]:
#test set
df_test['caption'].apply(nltk.word_tokenize)
df_test['caption'].head()

0       green-jeep-struggling-to-drive-over-huge-rocks
1    hiking-woman-tourist-is-walking-forward-in-mou...
2    close-up-of-african-american-doctors-hands-usi...
3    slow-motion-of-a-man-using-treadmill-in-the-gy...
4         slow-motion-of-photographer-in-national-park
Name: caption, dtype: object

In [0]:
# stemming the Word

porter = PorterStemmer()
lancaster=LancasterStemmer()

In [0]:
#dev set

def stem_sentences(sentence):
    tokens = sentence.split()
    #stemmed_tokens = [porter.stem(token) for token in tokens]
    stemmed_tokens = [lancaster.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)
  
df_dev['caption'] = df_dev['caption'].apply(stem_sentences)

In [0]:
#test set

def stem_sentences(sentence):
    tokens = sentence.split()
    #stemmed_tokens = [porter.stem(token) for token in tokens]
    stemmed_tokens = [lancaster.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)
  
df_test['caption'] = df_test['caption'].apply(stem_sentences)

In [0]:
df_dev.shape

(6000, 7)

In [0]:
df_test.shape

(2000, 3)

In [0]:
df_test

,video,caption,arrayInfo
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks,"[0.022039, 0.005264, 9.8e-05, 0.0, 0.003689, 0..."
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...,"[0.004694, 0.001934, 4e-05, 0.0, 0.001226, 5.8..."
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...,"[0.02653, 0.006208, 5.8e-05, 0.0, 0.00228, 3.1..."
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...,"[0.045109, 0.008317, 0.000151, 0.0, 0.005598, ..."
4,video7498.webm,slow-motion-of-photographer-in-national-park,"[0.034691, 0.008619, 0.000412, 0.0, 0.028355, ..."
...,...,...,...
1995,video10004.webm,astronaut-in-outer-space-against-the-backdrop-...,"[0.003161, 0.00033, 0.0, 0.0, 0.000189, 0.0, 0..."
1996,video10005.webm,young-women-lying-on-sunbed-and-applying-sun-c...,"[0.016932, 0.003384, 8.3e-05, 0.0, 0.001586, 6..."
1997,video10006.webm,doctor-talking-to-patient-using-a-tablet-to-ex...,"[0.029347, 0.008073, 0.000101, 0.0, 0.001891, ..."
1998,video10007.webm,businessman-sitting-on-the-beach-on-inflatable...,"[0.021746, 0.003096, 4.5e-05, 0.0, 0.002586, 9..."


In [0]:
# Concatenat the two datasets(dev and test) for generating the feature template
df = pd.concat([df_dev,df_test])
df.shape

(8000, 7)

In [0]:
df.head()

,video,caption,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,arrayInfo
0,video3.webm,blonde-woman-is-massaged-tilt-down,0.924,34.0,0.846,13.0,"[0.125563, 0.024036, 0.000314, 0.0, 0.015864, ..."
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot,0.923,33.0,0.667,12.0,"[0.007526, 0.001421, 6.8e-05, 0.0, 0.001184, 0..."
2,video6.webm,khr-gangsters,0.863,33.0,0.700,10.0,"[0.109584, 0.018978, 0.000289, 0.0, 0.008774, ..."
3,video8.webm,medical-helicopter-hovers-at-airport,0.922,33.0,0.818,11.0,"[0.120431, 0.013561, 0.000277, 0.0, 0.018974, ..."
4,video10.webm,couple-relaxing-on-picnic-crane-shot,0.950,34.0,0.900,10.0,"[0.005026, 0.001356, 5.5e-05, 0.0, 0.000665, 2..."


In [0]:
# Count Vectorizer
cv=CountVectorizer(ngram_range=(1, 2))

In [0]:
X_CV=cv.fit_transform(df['caption'])

In [0]:

type(X_CV)

scipy.sparse.csr.csr_matrix

In [0]:
#dev-set
X_arrseq=X_CV.toarray()
type(X_arrseq)

numpy.ndarray

In [0]:
print(X_arrseq[0])

[0 0 0 ... 0 0 0]


In [0]:
#Split the Data into Train and Test set

Y=df[['short-term_memorability','long-term_memorability']].values  #targets
X=X_arrseq #input



In [0]:
Y = Y[0:6000]
X = X_arrseq[0:6000]
Actual_x = X_arrseq[6000:]

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [197]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (4800, 31248)
X_test (1200, 31248)
Y_train (4800, 2)
Y_test (1200, 2)


In [198]:
Y

array([[0.924, 0.846],
       [0.923, 0.667],
       [0.863, 0.7  ],
       ...,
       [0.713, 0.818],
       [0.954, 1.   ],
       [0.953, 0.615]])

In [199]:
# Neural Networks Construction
n_cols = X_train.shape[1]
n_cols

31248

In [0]:
# Saving the number of columns in predictors: n_cols


# model Set up
model = Sequential()

# Adding the first layer
model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
model.add(Dropout(0.5))

# Adding the second layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

# Adding the output layer
model.add(Dense(2))

#Create a dense network. Use Early stopping monitor to avoid overfitting

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
   
# Defining early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)
    
# Fitting the model

history=model.fit(X_train, Y_train,epochs=30)


Epoch 1/3
4800/4800 [==============================] - 7s 2ms/step - loss: 0.1359 - accuracy: 0.5615
Epoch 2/3
4800/4800 [==============================] - 7s 1ms/step - loss: 0.0513 - accuracy: 0.5992
Epoch 3/3
4800/4800 [==============================] - 7s 1ms/step - loss: 0.0352 - accuracy: 0.6129


In [200]:
predictions1 = model.predict(X_test)
amit_caption = model.predict(Actual_x)
print(predictions1)
Get_score(predictions1, Y_test) 

[[0.85757184 0.75729614]
 [0.85894126 0.7725818 ]
 [0.8583276  0.77361333]
 ...
 [0.86580163 0.7724285 ]
 [0.8384506  0.7174108 ]
 [0.8390341  0.71910876]]
The Spearman's correlation coefficient is: 0.320
The Spearman's correlation coefficient is: 0.176


In [202]:
amit_caption

array([[0.81948787, 0.68366814],
       [0.83840936, 0.717247  ],
       [0.8666622 , 0.77557814],
       ...,
       [0.8628801 , 0.7493487 ],
       [0.8804043 , 0.81421125],
       [0.87826663, 0.79211825]], dtype=float32)

In [0]:
#Predict test set model 1
predictions1_test = model.predict(X_test)
print(predictions1_test)

[[0.82379854 0.7392753 ]
 [0.8005576  0.72046775]
 [0.78360784 0.7290983 ]
 ...
 [0.76948935 0.7205933 ]
 [0.7239307  0.6599254 ]
 [0.77115136 0.69373566]]


In [0]:
# TFIDF Algorithm
tfidf=TfidfVectorizer(min_df=1, stop_words='english')

In [0]:
X_TF=tfidf.fit_transform(df['caption'])

In [0]:

# test features
X_tfarrseq=X_TF.toarray()
X_tfarrseq

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
print(X_tfarrseq[0])

[0. 0. 0. ... 0. 0. 0.]


In [0]:

X=X_tfarrseq #input

In [0]:
#test set prediction

X_train = X_tfarrseq[0:6000]
X_test = X_tfarrseq[6000:]

In [0]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (6000, 5814)
X_test (2000, 5814)
Y_train (4800, 2)
Y_test (1200, 2)


In [0]:

n_cols = X_train.shape[1]
n_cols

5814

In [0]:

# Save the number of columns in predictors: n_cols


#  model Set up
model = Sequential()
# Adding the first layer
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
model.add(Dropout(0.5))
# Adding the second layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
# Adding the output layer
model.add(Dense(2))

In [0]:
# Compiling the model
model.compile(optimizer='adam', loss = 'mean_squared_error')

In [0]:
# Verify that model contains information from compiling
print("Loss function: " + model.loss)

Loss function: mean_squared_error


In [203]:
# Fitting the model

history=model.fit(X_train, Y_train,epochs=30)

Epoch 1/30
4800/4800 [==============================] - 4s 880us/step - loss: 0.0090
Epoch 2/30
4800/4800 [==============================] - 4s 863us/step - loss: 0.0089
Epoch 3/30
4800/4800 [==============================] - 4s 844us/step - loss: 0.0088
Epoch 4/30
4800/4800 [==============================] - 4s 850us/step - loss: 0.0088
Epoch 5/30
4800/4800 [==============================] - 4s 846us/step - loss: 0.0085
Epoch 6/30
4800/4800 [==============================] - 4s 835us/step - loss: 0.0083
Epoch 7/30
4800/4800 [==============================] - 4s 840us/step - loss: 0.0084
Epoch 8/30
4800/4800 [==============================] - 4s 830us/step - loss: 0.0085
Epoch 9/30
4800/4800 [==============================] - 4s 821us/step - loss: 0.0082
Epoch 10/30
4800/4800 [==============================] - 4s 818us/step - loss: 0.0081
Epoch 11/30
4800/4800 [==============================] - 4s 827us/step - loss: 0.0080
Epoch 12/30
4800/4800 [==============================] - 4s 914

In [205]:

predictions2 = model.predict(X_test)
print(predictions2)
Get_score(predictions2, Y_test) 

[[0.8865593  0.76809937]
 [0.8951937  0.8349497 ]
 [0.87158686 0.7799239 ]
 ...
 [0.8841008  0.7793247 ]
 [0.8549602  0.72832596]
 [0.8434168  0.69754404]]
The Spearman's correlation coefficient is: 0.378
The Spearman's correlation coefficient is: 0.170


In [206]:
#test set predictions
predictions2_test = model.predict(X_test)
print(predictions2_test)

[[0.8865593  0.76809937]
 [0.8951937  0.8349497 ]
 [0.87158686 0.7799239 ]
 ...
 [0.8841008  0.7793247 ]
 [0.8549602  0.72832596]
 [0.8434168  0.69754404]]


In [0]:
# IMAGE FEATURE MEMORABILITY
result_array = np.empty((0, 6075))
for line in df['arrayInfo']:
    result_array = np.append(result_array, np.array([line]), axis = 0)
    

In [253]:

X_arrHMP = result_array
print(type(X_arrHMP))
X_arrHMP

<class 'numpy.ndarray'>


array([[1.25563e-01, 2.40360e-02, 3.14000e-04, ..., 8.60000e-05,
        5.80000e-04, 0.00000e+00],
       [7.52600e-03, 1.42100e-03, 6.80000e-05, ..., 2.20000e-04,
        7.62000e-04, 1.22400e-03],
       [1.09584e-01, 1.89780e-02, 2.89000e-04, ..., 5.20000e-05,
        2.58000e-04, 2.15000e-04],
       ...,
       [2.93470e-02, 8.07300e-03, 1.01000e-04, ..., 2.62000e-04,
        3.54000e-04, 1.13000e-04],
       [2.17460e-02, 3.09600e-03, 4.50000e-05, ..., 1.20000e-04,
        1.29000e-04, 2.00000e-05],
       [1.12070e-02, 2.82700e-03, 1.37000e-04, ..., 2.85000e-04,
        4.49000e-04, 2.32000e-04]])

In [0]:

X=X_arrHMP[0:6000] #input
Y=df[['short-term_memorability','long-term_memorability']].values

In [0]:

#test set prediction

X_train = X[0:6000]
X_test = X[6000:]
Actual_HMP = X_arrHMP[6000:]

In [256]:
# Split the data
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42) 

print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (4800, 6075)
X_test (1200, 6075)
Y_train (4800, 2)
Y_test (1200, 2)


In [257]:
n_cols = X_train.shape[1]
n_cols

6075

In [0]:
#Add Dense Layers

model_1 = Sequential()
model_1.add(Dense(100, activation='relu', input_shape=(n_cols,)))
model_1.add(Dropout(0.5))
# Adding the second layer
model_1.add(Dense(100, activation='relu'))
model_1.add(Dropout(0.5))
# Adding the output layer
model_1.add(Dense(2))

In [0]:

# Compiling the model
model_1.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
   
# Defining early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)

In [260]:

# Fitting the model
history=model_1.fit(X_train, Y_train,epochs=30)



Epoch 1/30
4800/4800 [==============================] - 2s 481us/step - loss: 0.1118 - accuracy: 0.5775
Epoch 2/30
4800/4800 [==============================] - 2s 375us/step - loss: 0.0339 - accuracy: 0.6117
Epoch 3/30
4800/4800 [==============================] - 2s 363us/step - loss: 0.0271 - accuracy: 0.6388
Epoch 4/30
4800/4800 [==============================] - 2s 370us/step - loss: 0.0250 - accuracy: 0.6602
Epoch 5/30
4800/4800 [==============================] - 2s 389us/step - loss: 0.0232 - accuracy: 0.6762
Epoch 6/30
4800/4800 [==============================] - 2s 383us/step - loss: 0.0219 - accuracy: 0.6800
Epoch 7/30
4800/4800 [==============================] - 2s 367us/step - loss: 0.0214 - accuracy: 0.6919
Epoch 8/30
4800/4800 [==============================] - 2s 354us/step - loss: 0.0213 - accuracy: 0.6998
Epoch 9/30
4800/4800 [==============================] - 2s 347us/step - loss: 0.0208 - accuracy: 0.7015
Epoch 10/30
4800/4800 [==============================] - 2s 336u

In [261]:
# for training set
predictions3 = model_1.predict(X_test)
Amit_HMP = model_1.predict(Actual_HMP)
print(predictions3)
Get_score(predictions3, Y_test)

[[0.8838241  0.8066225 ]
 [0.8525182  0.774944  ]
 [0.88458323 0.8077444 ]
 ...
 [0.8745872  0.79708517]
 [0.86983156 0.79242784]
 [0.8594893  0.78209317]]
The Spearman's correlation coefficient is: 0.285
The Spearman's correlation coefficient is: 0.121


In [0]:
labels_test[['short-term_memorability','long-term_memorability']] = Amit_HMP

In [220]:
labels_test

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,0.836993,33,0.753683,12
1,7495,0.848990,34,0.765504,10
2,7496,0.878214,32,0.794203,13
3,7497,0.855277,33,0.771780,10
4,7498,0.823131,33,0.740187,10
...,...,...,...,...,...
1995,10004,0.858914,34,0.775164,17
1996,10005,0.857334,34,0.773682,9
1997,10006,0.857410,34,0.773685,12
1998,10007,0.840250,34,0.756929,12


In [0]:
labels_test.to_csv('/content/drive/My Drive/Test_Results1.csv',index=None)

In [182]:
print(predictions_test)

[[1.728823  1.4667335]
 [1.7434413 1.5805445]
 [1.71924   1.5661911]
 ...
 [1.7527041 1.5598409]
 [1.6931514 1.4187878]
 [1.6828434 1.404185 ]]


In [0]:
predictions_test = np.divide(predictions_test, 3)

In [223]:
predictions_test.shape

(1200, 2)

In [224]:
#Final Predictions with Ensemble
print(predictions_test)

[[0.57627434 0.48891115]
 [0.58114713 0.52684814]
 [0.57308    0.5220637 ]
 ...
 [0.5842347  0.519947  ]
 [0.5643838  0.47292927]
 [0.56094784 0.4680617 ]]


In [0]:
#Extract the results in a csv
df_test_results = pd.DataFrame(columns = ['video', 'short-term', 'long-term'])
df_test_results['short-term'] = predictions_test[:,0]
df_test_results['long-term'] = predictions_test[:,1]
df_test_results['video'] = df['video'][6000:]

In [226]:
df_test_results.head()

,video,short-term,long-term
0,video7494.webm,0.576274,0.488911
1,video7495.webm,0.581147,0.526848
2,video7496.webm,0.573080,0.522064
3,video7497.webm,0.537063,0.465260
4,video7498.webm,0.564899,0.487297


In [0]:


df_test_results.to_csv('/content/drive/My Drive/Test_Results.csv',index=None)


In [228]:

print(predictions3)
Get_score(predictions3, Y_test)


[[0.86663616 0.7828808 ]
 [0.84350204 0.7600992 ]
 [0.86958027 0.7855265 ]
 ...
 [0.8597808  0.776061  ]
 [0.85609335 0.77241904]
 [0.8481791  0.7646832 ]]
The Spearman's correlation coefficient is: 0.289
The Spearman's correlation coefficient is: 0.127


In [0]:
#Creating an ensemble with Weighted Average method
predictions_weighted  = 0.40*predictions1 + 0.40*predictions2 + 0.10*predictions3

In [231]:
print(predictions_weighted)

[[0.78431606 0.6884463 ]
 [0.78600425 0.7190226 ]
 [0.77892387 0.69996756]
 ...
 [0.78593904 0.6983074 ]
 [0.76297367 0.65553665]
 [0.75779825 0.64312947]]


In [232]:

print(predictions_weighted)
Get_score(predictions_weighted, Y_test) 

[[0.78431606 0.6884463 ]
 [0.78600425 0.7190226 ]
 [0.77892387 0.69996756]
 ...
 [0.78593904 0.6983074 ]
 [0.76297367 0.65553665]
 [0.75779825 0.64312947]]
The Spearman's correlation coefficient is: 0.386
The Spearman's correlation coefficient is: 0.181
